<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/JNCLectures_Intro_to_ML/blob/main/Week11/NN_comp_graph_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

In [2]:
x = np.array([ [0, 1], [1, 0], [1, 1],[0, 0] ]).T
y = np.array([[1], [1], [0], [0]]).T
print(x.shape,y.shape)

(2, 4) (1, 4)


In [3]:
num_input = 2
num_hidden = 5
num_output = 1

In [4]:
W1_tf = tf.Variable(initial_value = tf.random.uniform(shape = (num_input, num_hidden)))
W2_tf = tf.Variable(initial_value = tf.random.uniform(shape = (num_hidden, num_output)))

In [5]:
def model(inputs):
  return tf.sigmoid(tf.matmul(tf.sigmoid(tf.matmul(inputs, W1_tf)), W2_tf))

In [6]:
x = x.astype(np.float32)
y = y.astype(np.float32)

In [7]:
def square_loss(targets, predictions):
  per_sample_losses = tf.square(targets - predictions)
  return tf.reduce_mean(per_sample_losses)

In [ ]:
# prtevious metrod for training step

learning_rate = 0.01

def training_step(inputs, targets):
  with tf.GradientTape(persistent=True) as tape:
    predictions = model(inputs)
    loss = square_loss(predictions, targets)
  grad_loss_wrt_W1 = tape.gradient(loss, W1_tf)
  grad_loss_wrt_W2 = tape.gradient(loss, W2_tf)
  W1_tf.assign_sub(grad_loss_wrt_W1 * learning_rate)
  W2_tf.assign_sub(grad_loss_wrt_W2 * learning_rate)

  return loss


In [8]:
# breakdown to see derivatives
def training_step(inputs, targets):
  with tf.GradientTape(persistent=True) as tape:
      z1_ = tf.matmul(inputs, W1_tf)
      h1_ = tf.sigmoid(z1_)
      z2_ = tf.matmul(h1_, W2_tf)
      predictions = tf.sigmoid(z2_)
      loss = square_loss(predictions, targets)

  # calculate individual gradients in the graph
  dL_dh2 = tape.gradient(loss, predictions)
  dh2_dz2 = tape.gradient(predictions, z2_)
  dz2_dw2 = tape.gradient(z2_, W2_tf)
  dz2_dh1 = tape.gradient(z2_, h1_)
  dh1_dz1 = tape.gradient(h1_, z1_)
  dz1_dw1 = tape.gradient(z1_, W1_tf)
  derivatives = [dL_dh2, dh2_dz2, dz2_dw2, dz2_dh1, dh1_dz1, dz1_dw1]

  # calculate weight updates
  grad_loss_wrt_W1 = tape.gradient(loss, W1_tf)
  grad_loss_wrt_W2 = tape.gradient(loss, W2_tf)
  W1_tf.assign_sub(grad_loss_wrt_W1 * learning_rate)
  W2_tf.assign_sub(grad_loss_wrt_W2 * learning_rate)

  return derivatives, grad_loss_wrt_W2, grad_loss_wrt_W1


In [10]:
learning_rate = 0.01
num_iterations = 2
for step in range(num_iterations):
  derivatives, dL_dW2, dL_dW1 = training_step(inputs = x.T, targets = y)
  print(derivatives, dL_dW2, dL_dW1)


[<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.19750029],
       [0.20682791],
       [0.21692392],
       [0.17649743]], dtype=float32)>, <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.09397455],
       [0.07888886],
       [0.06177605],
       [0.12539463]], dtype=float32)>, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[2.6203775],
       [2.6067884],
       [2.5797248],
       [2.089467 ],
       [2.7633436]], dtype=float32)>, <tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.31020162, 0.977025  , 0.6835227 , 0.7227717 , 0.82305115],
       [0.31020162, 0.977025  , 0.6835227 , 0.7227717 , 0.82305115],
       [0.31020162, 0.977025  , 0.6835227 , 0.7227717 , 0.82305115],
       [0.31020162, 0.977025  , 0.6835227 , 0.7227717 , 0.82305115]],
      dtype=float32)>, <tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.23903643, 0.2427084 , 0.2360135 , 0.24897373, 0.20996463],
       [0.20200005, 0.19735566, 0.21626687, 0.24983783, 0.20533271],